# Fine Tuning TinyBert

In [47]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score

## 🚀 1. Carregar dataset de exemplo

In [48]:
dataset = load_dataset("imdb")

KeyboardInterrupt: 

In [46]:
print(dataset["train"][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## 🚀 2. Carregar Tokenizer e Modelo TinyBERT pré-treinado

In [9]:
    model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 🚀 3. Tokenizar os textos

In [10]:
def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=128)

In [11]:
dataset = dataset.map(tokenize, batched=True)

In [45]:
print(dataset.column_names)

{'train': ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'unsupervised': ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']}


## 🚀 4. Definir métricas de avaliação

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

## 🚀 5. Configuração do Treinamento

In [13]:
training_args = TrainingArguments(
    output_dir="./tinybert_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10
)

C:\Users\victo\PycharmProjects\tinybert-fine-tuning\.venv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 🚀 6. Treinar o Modelo

### Verificando se há GPU esta disponivel

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("GPU disponível:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma GPU detectada")

GPU disponível: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
model.to(device)
trainer.train()

C:\Users\victo\AppData\Local\Temp\ipykernel_16088\3848091158.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.267300,0.591842,0.815640
2,0.356700,0.611452,0.846360
3,0.208700,0.722655,0.851920


TrainOutput(global_step=18750, training_loss=0.43909049767156444, metrics={'train_runtime': 499.7498, 'train_samples_per_second': 150.075, 'train_steps_per_second': 37.519, 'total_flos': 268856179200000.0, 'train_loss': 0.43909049767156444, 'epoch': 3.0})

## 🚀 7. Testar o Modelo Treinado

In [52]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}  # Move inputs para o mesmo dispositivo do modelo
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "Positivo" if prediction == 1 else "Negativo"

print("Exemplo de previsão:", predict("I don't love this movie"))

Exemplo de previsão: Negativo


# all-MiniLM-L12-v2 Fine Tuning IMDB

In [55]:
model_name = "sentence-transformers/all-MiniLM-L12-v2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L12-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./all_mini_L12_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

C:\Users\victo\AppData\Local\Temp\ipykernel_16088\51941973.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.227000,0.503391,0.853280
2,0.481500,0.523052,0.864960


In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}  # Move inputs para o mesmo dispositivo do modelo
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "Positivo" if prediction == 1 else "Negativo"

print("Exemplo de previsão:", predict("I don't love this movie"))